<a href="https://colab.research.google.com/github/maengg/Customer_churn/blob/main/Customer_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **데이터 설명**

*데이터 셋은 kaggle의 Telco Customer Churn 데이터셋으로 3분기에 캘리포니아의 7043 고객에게 집 전화 및 인터넷 서비스를 제공한 가상의 통신 회사에 대한 정보를 담고 있다. (데이터의 형태는 7043 rows, 21 columns)*

<br>

### **각 column 설명**

* gender : 고객의 성별
* SeniorCitizen : 고객이 65세 이상인지 아닌지 (Yes : 1, No : 0)
* Dependents : 고객이 부양가족과 함께 살고 있는지, 부양가족은 자녀, 부모, 조부모 등이 될 수 있다. (Yes , No)
* Partner : 결혼 여부(Yes ,No)
* tenure : 누적 이용 기간 (단위 : 월)
* PhoneService : 고객이 집 전화 서비스에 구독했는지 여부 (Yes ,No)
* MultipleLines : 고객이 여러 전화선에 구독했는지 여부 (Yes, No, No phone service)
* InternetService : 고객이 인터넷 서비스에 구독했는지 여부 (No, DSL, Fiber optic)
* OnlineSecurity : 고객이 온라인 보안 서비스에 구독했는지 여부 (Yes, No, No internet service)
* OnlineBackup : 고객이 온라인 백업 서비스를 구독했는지 여부 (Yes, No, No internet service)
* DeviceProtection : 고객이 추가 장치 보호 요금제에 가입했는지 여부 (Yes, No, No internet service)
* TechSupport : 고객이 줄어든 대기시간으로 추가 기술 지원 요금제에 가입했는지 여부 (Yes, No, No internet service)
* StreamingTV : 고객이 인터넷을 사용하여 타사 TV 스트리밍을 이용하는지 여부 (Yes, No, No internet service) 이용하는 경우 회사에서 추가 요금 요구하지 않음.
* StreamingMovies : 고객이 인터넷을 사용하여 타사 영화 스트리밍을 이용하는지 여부 (Yes, No, No internet service) 이용하는 경우 회사에서 추가 요금 요구하지 않음.
* Contract : 고객의 현재 계약 유형 (Month-to-month(월간), One year(1년), Two year(2년))
* PaperlessBilling : 고객이 종이 없는 청구서를 선택했는지
* PaymentMethod : 고객이 청구서를 지불하는 방법 (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))
* MonthlyCharges : 월별 납부 금액
* TotalCharges : 누적 납부 금액
* Churn : 이탈 여부 (Yes, No), Target data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 라이브러리 불러오기.
import pandas as pd

In [4]:
df = pd.read_csv('/content/drive/MyDrive/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [5]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
